# CHATBOTS - Using Natural Language Processing and Tensorflow

In this Jupyter Notebook, We are going to Build a Chatbot that Understands the Context of Sentense and Respond accordingly.
These are the Things that we are going to do in this Project -
1. Transforming the Conversational Intents into Tensorflow model (Neural Network using TFLEARN) using NLP and Save it as Pickle also.
2. Load the Same Pickle and Model to Build the Framework to Process the Responses.
3. At Last, We Show How the Inputs are Processed and Give the Reponses.
-------------------------------------------------------------------------------------------------------

##### TFLEARN  - TFlearn is a modular and transparent deep learning library built on top of Tensorflow. It was designed to provide a higher-level API to TensorFlow in order to facilitate and speed-up experimentations, while remaining fully transparent and compatible with it.  (http://tflearn.org/)
-------------------------------------------------------------------------------------------------------
##### TENSORFLOW - TensorFlow is an end-to-end open source platform for machine learning. It has a comprehensive, flexible ecosystem of tools, libraries and community resources that lets researchers push the state-of-the-art in ML and developers easily build and deploy ML powered applications.


In [1]:
from IPython.core.display import Image, display
# display(Image('Untitled.png'))

Libraries ------------------------------------------------------------------------------#FreeBirdsCrew

In [4]:
pip install tflearn 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 107 kB 25.9 MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=1b499fa57dbc94a5b282557a0e20aaa3a300ae7898f121fde16f8e7e2fe42fe3
  Stored in directory: /root/.cache/pip/wheels/65/9b/15/cb1e6b279c14ed897530d15cfd7da8e3df8a947e593f5cfe59
Successfully built tflearn


In [2]:
#Used in Tensorflow Model
import numpy as np
import tensorflow as tf
import tflearn
import random

#Usde to for Contextualisation and Other NLP Tasks.
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

#Other
import json
import pickle
import warnings
warnings.filterwarnings("ignore")

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
print("Processing the Intents.....")
with open('intents.json') as json_data:
    intents = json.load(json_data)

Processing the Intents.....


In [4]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi',
    'How are you',
    'Is anyone there?',
    'Hello',
    'Good day',
    'Arwaaah'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye', 'saha'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'Studying sunday',
   'patterns': ['what we have on sunday',
    'what we going to study on sunday',
    'Sunday'],
   'responses': ['you have AAV at 9h30, OCO at 11h']},
  {'tag': 'Studying Monday',
   'patterns': ['what we have on monday',
    'what we going to study on monday',
    'Monday'],
   'responses': ['you have AAV at 8h, productique at 11h, TP rpi at 13:30']},
  {'tag': 'Studying tuesday',
   'patterns': ['what we have on tuesday',
    'what we going to study on tuesday',
    'tuesday'],
   'responses'

In [5]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
words = []
classes = []
documents = []
ignore_words = ['?']
print("Looping through the Intents to Convert them to words, classes, documents and ignore_words.......")
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

Looping through the Intents to Convert them to words, classes, documents and ignore_words.......


In [7]:
print("Stemming, Lowering and Removing Duplicates.......")
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

Stemming, Lowering and Removing Duplicates.......
61 documents
20 classes ['Module', 'Studying Monday', 'Studying sunday', 'Studying thursday', 'Studying tuesday', 'Studying wednesday', 'TP', 'TP2', 'deadline', 'goodbye', 'greeting', 'help', 'homework', 'hours', 'interro OCO', 'interro productique', 'motivation', 'rental', 'thanks', 'today']
76 unique stemmed words ["'d", "'s", 'a', 'aav', 'any', 'anyon', 'ar', 'arwaaah', 'automat', 'avancé', 'backstep', 'bixby', 'bye', 'ca', 'can', 'cant', 'day', 'deadlin', 'do', 'doe', 'for', 'forward', 'going', 'good', 'goodby', 'hat', 'hav', 'hello', 'help', 'hi', 'homework', 'hour', 'how', 'i', 'is', 'lat', 'lik', 'me', 'monday', 'mop', 'mot', "n't", 'near', 'nee', 'oco', 'on', 'op', 'press', 'product', 'reg', 'rent', 'rpi', 'sah', 'see', 'study', 'studyund', 'sunday', 'test', 'thank', 'that', 'ther', 'thi', 'thursday', 'to', 'today', 'tomorrow', 'tp1', 'tuesday', 'we', 'wednesday', 'what', 'when', 'wher', 'work', 'yo', 'you']


In [8]:
print("Creating the Data for our Model.....")
training = []
output = []
print("Creating an List (Empty) for Output.....")
output_empty = [0] * len(classes)

print("Creating Traning Set, Bag of Words for our Model....")
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

Creating the Data for our Model.....
Creating an List (Empty) for Output.....
Creating Traning Set, Bag of Words for our Model....


In [9]:
training

[[[0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
 [[0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1],
  [0, 0, 0, 0, 0, 0, 

In [10]:
from tensorflow.python.framework import ops

In [11]:
print("Shuffling Randomly and Converting into Numpy Array for Faster Processing......")
random.shuffle(training)
training = np.array(training)

print("Creating Train and Test Lists.....")
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Building Neural Network for Out Chatbot to be Contextual....")
print("Resetting graph data....")
ops.reset_default_graph()

Shuffling Randomly and Converting into Numpy Array for Faster Processing......
Creating Train and Test Lists.....
Building Neural Network for Out Chatbot to be Contextual....
Resetting graph data....


In [12]:
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)
print("Training....")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Training....


In [13]:
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

In [14]:
print("Training the Model.......")
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
print("Saving the Model.......")
model.save('model.tflearn')


Training Step: 7999  | total loss: 0.03045 | time: 0.024s
| Adam | epoch: 1000 | loss: 0.03045 - acc: 0.9771 -- iter: 56/61
Training Step: 8000  | total loss: 0.02785 | time: 0.027s
| Adam | epoch: 1000 | loss: 0.02785 - acc: 0.9794 -- iter: 61/61
--
Saving the Model.......


In [15]:
print("Pickle is also Saved..........")
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

Pickle is also Saved..........


In [16]:
print("Loading Pickle.....")
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']


with open('intents.json') as json_data:
    intents = json.load(json_data)
    
print("Loading the Model......")
# load our saved model
model.load('./model.tflearn')

Loading Pickle.....
Loading the Model......


In [17]:
def clean_up_sentence(sentence):
    # It Tokenize or Break it into the constituents parts of Sentense.
    sentence_words = nltk.word_tokenize(sentence)
    # Stemming means to find the root of the word.
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# Return the Array of Bag of Words: True or False and 0 or 1 for each word of bag that exists in the Sentence
def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

ERROR_THRESHOLD = 0.25
print("ERROR_THRESHOLD = 0.25")

def classify(sentence):
    # Prediction or To Get the Posibility or Probability from the Model
    results = model.predict([bow(sentence, words)])[0]
    # Exclude those results which are Below Threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # Sorting is Done because heigher Confidence Answer comes first.
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1])) #Tuppl -> Intent and Probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # That Means if Classification is Done then Find the Matching Tag.
    if results:
        # Long Loop to get the Result.
        while results:
            for i in intents['intents']:
                # Tag Finding
                if i['tag'] == results[0][0]:
                    # Random Response from High Order Probabilities
                    return print(random.choice(i['responses']))

            results.pop(0)

ERROR_THRESHOLD = 0.25


In [ ]:
while True:
    input_data = input("You- ")
    answer = response(input_data)
    answer

You- help me on Forwarding
you can check this youtube channel: NON-LINEAR systrem controll
You- help me on Forwarding
you can check this youtube channel: NON-LINEAR systrem controll
You- help me on Forwarding
you can check this youtube channel: NON-LINEAR systrem controll
You- where can I study Backstepping
check the book of KHALIL
You- where can I study Backstepping
you can check this youtube channel: NON-LINEAR systrem controll
You- help me Forwarding
check the book of KHALIL
You- TP1
first january, you still have time
You- tommorow
You- help me
How can I help you
You- i hate studying
watch this reel u will be happy
You- i need motivation
lets play a game after go to study
You- arwaaah
Hello, thanks for visiting


KeyboardInterrupt: ignored